## Check the setup and connect to the database

In [ ]:
%run '010-check_setup.ipynb'

# Tables from SAP HANA

In [ ]:
column_id="PassengerId"
column_label="Survived"
features_subset=[
            "NameFirstLetter",
            "FarePerPerson",
            "SibSp",
            "ParCh",
            "Gender",
            "PClass",
            "Embarked",
            "Title",
            "Age",
            "PersonsPerTicket"
]

In [ ]:
hdf_titanic_train = myconn.table("TRAIN_FE").select(
            [column_id]+features_subset+[column_label]
        ).cast(column_label, 'NVARCHAR(1)')

# Model training with different [Hyperparameters](https://en.wikipedia.org/wiki/Hyperparameter_(machine_learning))

In [ ]:
import hana_ml.algorithms.pal.unified_classification as hml_uc

In [ ]:
rdt_params = dict(random_state=2,
                  min_samples_leaf=10,
                  n_estimators=5,
                  max_depth=2)

In [ ]:
uc_rdt_v6 = hml_uc.UnifiedClassification(func='RandomDecisionTree', **rdt_params)

In [ ]:
uc_rdt_v6.fit(
    data=hdf_titanic_train,
    key='PassengerId', label='Survived',
    training_percent=0.8,
    partition_method='stratified', stratified_column='Survived', 
    partition_random_state=2
);

In [ ]:
uc_rdt_v6.get_parameters()

## Generate a model report

In [ ]:
from hana_ml.visualizers.unified_report import UnifiedReport
UnifiedReport(uc_rdt_v6).build().display()

## Debrief the model

In [ ]:
from hana_ml.visualizers.model_debriefing import TreeModelDebriefing

In [ ]:
TreeModelDebriefing.tree_debrief_with_dot(uc_rdt_v6.model_[0]);

# Store the model

In [ ]:
from hana_ml.model_storage import ModelStorage

In [ ]:
ms = ModelStorage(myconn)

In [ ]:
uc_rdt_v6.name = 'CodeJam-Titanic-Classification'
uc_rdt_v6.version = 6

In [ ]:
ms.save_model(model=uc_rdt_v6, if_exists='replace', save_report=False)

In [ ]:
ms.list_models()

# Call prediction

In [ ]:
hdf_titanic_test=myconn.table('TEST_FE').select(
            [column_id]+features_subset
        )

In [ ]:
hdf_res = uc_rdt_v6.predict(hdf_titanic_test, key = 'PassengerId')

In [ ]:
display(hdf_res.select('PassengerId', 'SCORE', 'CONFIDENCE').head(4).collect())

## Global Interpretation using [Shapley values](https://en.wikipedia.org/wiki/Shapley_value#In_machine_learning)

Create Shapley explainer to explain the output of machine learning model: https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2024_1_QRC/en-US/hana_ml.visualizers.html#hana_ml.visualizers.model_debriefing.TreeModelDebriefing.shapley_explainer

In [ ]:
from hana_ml.visualizers.model_debriefing import TreeModelDebriefing

In [ ]:
shapley_explainer = TreeModelDebriefing.shapley_explainer(hdf_res, hdf_titanic_test, key='PassengerId', label='Survived')
shapley_explainer.summary_plot()